In [17]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [18]:
!pip install catboost


In [19]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import joblib
from google.colab import drive

In [20]:
data = pd.read_csv('/content/drive/MyDrive/DiseaseAndSymptoms.csv')
print(data.head())
print(data.columns)

            Disease   Symptom_1              Symptom_2              Symptom_3  \
0  Fungal infection     itching              skin_rash   nodal_skin_eruptions   
1  Fungal infection   skin_rash   nodal_skin_eruptions    dischromic _patches   
2  Fungal infection     itching   nodal_skin_eruptions    dischromic _patches   
3  Fungal infection     itching              skin_rash    dischromic _patches   
4  Fungal infection     itching              skin_rash   nodal_skin_eruptions   

              Symptom_4 Symptom_5 Symptom_6 Symptom_7 Symptom_8 Symptom_9  \
0   dischromic _patches       NaN       NaN       NaN       NaN       NaN   
1                   NaN       NaN       NaN       NaN       NaN       NaN   
2                   NaN       NaN       NaN       NaN       NaN       NaN   
3                   NaN       NaN       NaN       NaN       NaN       NaN   
4                   NaN       NaN       NaN       NaN       NaN       NaN   

  Symptom_10 Symptom_11 Symptom_12 Symptom_13 Symp

In [21]:
# Fill missing values with 'none'
data = data.fillna('none')


In [22]:
# Initialize vectorizer
vectorizer = CountVectorizer()

In [24]:
# Combine symptoms into a single string for vectorization
data['combined_symptoms'] = data['Symptom_1'] + ' ' + data['Symptom_2'] + ' ' + data['Symptom_3'] + ' ' + data['Symptom_4']

# Vectorize symptoms
X = vectorizer.fit_transform(data['combined_symptoms'])
y = data['Disease']

In [25]:
model = CatBoostClassifier()

In [26]:
param_grid = {
    'iterations': [100, 200],
    'depth': [4, 6],
    'learning_rate': [0.01, 0.1],
}

In [27]:
# Extract features and target variable
X = data.loc[:, 'Symptom_1':'Symptom_4']
y = data['Disease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:


random_search = RandomizedSearchCV(model, param_distributions=param_grid, scoring='accuracy', cv=2, verbose=1, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train, cat_features=['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4'])


Fitting 2 folds for each of 8 candidates, totalling 16 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


0:	learn: 3.0990596	total: 306ms	remaining: 1m
1:	learn: 2.6771496	total: 643ms	remaining: 1m 3s
2:	learn: 2.3907147	total: 947ms	remaining: 1m 2s
3:	learn: 2.1521554	total: 1.29s	remaining: 1m 3s
4:	learn: 1.9798780	total: 1.64s	remaining: 1m 4s
5:	learn: 1.8735671	total: 2.01s	remaining: 1m 4s
6:	learn: 1.7305529	total: 2.36s	remaining: 1m 4s
7:	learn: 1.6046221	total: 2.73s	remaining: 1m 5s
8:	learn: 1.4960420	total: 3.11s	remaining: 1m 6s
9:	learn: 1.4179254	total: 3.5s	remaining: 1m 6s
10:	learn: 1.3039255	total: 3.9s	remaining: 1m 6s
11:	learn: 1.2362429	total: 4.28s	remaining: 1m 7s
12:	learn: 1.1584691	total: 4.67s	remaining: 1m 7s
13:	learn: 1.0692175	total: 5.09s	remaining: 1m 7s
14:	learn: 1.0042361	total: 5.49s	remaining: 1m 7s
15:	learn: 0.9394281	total: 5.87s	remaining: 1m 7s
16:	learn: 0.8875991	total: 6.3s	remaining: 1m 7s
17:	learn: 0.8464617	total: 6.68s	remaining: 1m 7s
18:	learn: 0.8028619	total: 7.09s	remaining: 1m 7s
19:	learn: 0.7603898	total: 7.51s	remaining: 1m

RandomizedSearchCV(cv=2,
                   estimator=<catboost.core.CatBoostClassifier object at 0x799ad7195f00>,
                   n_jobs=-1,
                   param_distributions={'depth': [4, 6],
                                        'iterations': [100, 200],
                                        'learning_rate': [0.01, 0.1]},
                   random_state=42, scoring='accuracy', verbose=1)

In [29]:
best_params = random_search.best_params_
print(f'Best parameters: {best_params}')

Best parameters: {'learning_rate': 0.1, 'iterations': 200, 'depth': 6}


In [31]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9949186991869918


In [32]:
# Save the model and vectorizer
model_path = '/content/drive/MyDrive/Chatbot/best_model.joblib'
vectorizer_path = '/content/drive/MyDrive/Chatbot/vectorizer.joblib'
random_search_path = '/content/drive/MyDrive/Chatbot/random_search.joblib'

joblib.dump(best_model, model_path)
joblib.dump(vectorizer, vectorizer_path)
joblib.dump(random_search, random_search_path)

print(f'Model saved to {model_path}')
print(f'Vectorizer saved to {vectorizer_path}')
print(f'Random search object saved to {random_search_path}')

Model saved to /content/drive/MyDrive/Chatbot/best_model.joblib
Vectorizer saved to /content/drive/MyDrive/Chatbot/vectorizer.joblib
Random search object saved to /content/drive/MyDrive/Chatbot/random_search.joblib
